In [40]:
import DataGenerator
import ShipDetection
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Input
from keras.optimizers import Adam
import pandas as pd
import glob
import tensorflow as tf
from keras.models import load_model
import matplotlib.pyplot as plt

Set local variables

In [32]:
imgWidth = 256
imgHeight = 256
batch_size = 8
trainDir = "data/train/"
validDir = "data/validation/"

Set paths to data

In [33]:
trainPaths = glob.glob(trainDir + "*.jpg")
validationPaths = glob.glob(validDir + "*.jpg")

In [34]:
train_ship_segmentations_v2 = pd.read_csv("train_ship_segmentations_v2.csv")
train_ship_segmentations_v2.set_index("ImageId", inplace=True)

Create Data generators for model

In [35]:
with tf.device("/CPU:0"):
    training_generator = DataGenerator.DataGenerator(trainPaths,
                                                     train_ship_segmentations_v2,
                                                     batch_size=batch_size)
    validation_generator = DataGenerator.DataGenerator(validationPaths,
                                                       train_ship_segmentations_v2,
                                                       batch_size=batch_size)

Create model

In [26]:
with tf.device("/CPU:0"):
    inputImg = Input((imgWidth, imgHeight, 3), name='img')
    model = ShipDetection.get_unet(inputImg)
    model.compile(optimizer=Adam(lr=0.01), loss=ShipDetection.dice_coef_loss,metrics=['accuracy'])

D:\AdankoProject\python\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_39 (Conv2D)             (None, 256, 256, 16  448         ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 batch_normalization_37 (BatchN  (None, 256, 256, 16  64         ['conv2d_39[0][0]']              
 ormalization)                  )                                                           

Set fitting callbacks

In [36]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model1', verbose=1, save_best_only=True, save_weights_only=True)
]

Fit model

In [37]:
model = load_model("modelFinal1", custom_objects = {"dice_coef_loss":ShipDetection.dice_coef_loss})

In [38]:
with tf.device("/CPU:0"):
    results = model.fit(training_generator,
                        validation_data=validation_generator,
                        epochs=5,
                        callbacks=callbacks,
                        verbose=1
                        )

Epoch 1/5
2550/2550 [==============================] - ETA: 0s - loss: 0.9862 - accuracy: 0.9661
Epoch 1: val_loss improved from inf to 0.99381, saving model to model1
2550/2550 [==============================] - 7331s 3s/step - loss: 0.9862 - accuracy: 0.9661 - val_loss: 0.9938 - val_accuracy: 0.9727 - lr: 0.0100
Epoch 2/5
2550/2550 [==============================] - ETA: 0s - loss: 0.9859 - accuracy: 0.9647
Epoch 2: val_loss improved from 0.99381 to 0.99380, saving model to model1
2550/2550 [==============================] - 7130s 3s/step - loss: 0.9859 - accuracy: 0.9647 - val_loss: 0.9938 - val_accuracy: 0.9819 - lr: 0.0100
Epoch 3/5
2550/2550 [==============================] - ETA: 0s - loss: 0.9857 - accuracy: 0.9653
Epoch 3: val_loss did not improve from 0.99380
2550/2550 [==============================] - 7258s 3s/step - loss: 0.9857 - accuracy: 0.9653 - val_loss: 0.9940 - val_accuracy: 0.9935 - lr: 0.0100
Epoch 4/5
2550/2550 [==============================] - ETA: 0s - loss: 0

In [39]:
model.save("modelFinal1")

INFO:tensorflow:Assets written to: modelFinal1\assets


INFO:tensorflow:Assets written to: modelFinal1\assets


In [ ]:
plt.plot(results.history)